In [ ]:
!pip install flask pyngrok tweepy transformers pandas flask-cors

In [ ]:
from google.colab import userdata

In [ ]:
import io
import pandas as pd
from flask import Flask, request, jsonify
import tweepy
from transformers import pipeline
from pyngrok import ngrok, conf
from flask_cors import CORS
from langdetect import detect
import chardet

In [ ]:

pd.set_option('display.max_colwidth', None)

conf.get_default().auth_token = userdata.get('NGROK')


api_key = "Your_API_Key"
api_key_secret = "Your_API_Key_Secret"
access_token = "Your_Access_token"
access_token_secret = "Your_Access_token_secret"
bearer_token = "Your_bearear_token"

client = tweepy.Client(
    bearer_token=bearer_token,
    consumer_key=api_key,
    consumer_secret=api_key_secret,
    access_token=access_token,
    access_token_secret=access_token_secret
)

sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-xlm-roberta-base-sentiment")
emotion_pipeline = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=False)

def get_category(sentiment):
    label = sentiment['label']
    if label == "negative":
        return "negative"
    elif label == "neutral":
        return "neutral"
    else:
        return "positive"

app = Flask(__name__)
CORS(app)

@app.route('/analyze_query', methods=['POST'])
def analyze_query():
    results = []
    tweet_texts = []

    data = request.get_json(silent=True)
    user_query = data.get("query", None) if data else request.values.get("query")

    if not user_query:
        return jsonify({"error": "No query provided"}), 400

    language = data.get("language", "en") if data else "en"
    query = f"{user_query} -is:retweet lang:{language}"

    try:
        response = client.search_recent_tweets(query=query, max_results=10)
        tweets = response.data if response.data is not None else []
    except Exception as e:
        return jsonify({"error": "Error fetching tweets: " + str(e)}), 500

    if not tweets:
        return jsonify({"error": "No tweets found for the query"}), 404

    tweet_texts = [tweet.text for tweet in tweets]

    for tweet_text in tweet_texts:
        detected_lang = detect(tweet_text)
        sentiment_result = sentiment_pipeline(tweet_text)[0]
        category = get_category(sentiment_result)
        emotion_result = emotion_pipeline(tweet_text)[0]

        results.append({
            "Tweet": tweet_text,
            "Detected Language": detected_lang,
            "Sentiment": category,
            "Emotion": emotion_result['label'],
            "Emotion Score": round(emotion_result['score'], 3)
        })

    return jsonify({"results": results})

@app.route('/analyze_file', methods=['POST'])
def analyze_file():
    results = []
    tweet_texts = []

    file = request.files.get("file", None)
    if not file:
        return jsonify({"error": "No CSV file uploaded"}), 400

    try:
        file_bytes = file.read()
        detected_encoding = chardet.detect(file_bytes)["encoding"]
        df_csv = pd.read_csv(io.BytesIO(file_bytes), encoding=detected_encoding or "utf-8", encoding_errors="replace")

        tweet_texts = df_csv['Tweet'].tolist() if 'Tweet' in df_csv.columns else df_csv.iloc[:, 0].tolist()
    except Exception as e:
        return jsonify({"error": f"Failed to process CSV file: {str(e)}"}), 500

    for tweet_text in tweet_texts:
        detected_lang = detect(tweet_text)
        sentiment_result = sentiment_pipeline(tweet_text)[0]
        category = get_category(sentiment_result)
        emotion_result = emotion_pipeline(tweet_text)[0]

        results.append({
            "Tweet": tweet_text,
            "Detected Language": detected_lang,
            "Sentiment": category,
            "Emotion": emotion_result['label'],
            "Emotion Score": round(emotion_result['score'], 3)
        })

    return jsonify({"results": results})

public_url = ngrok.connect(5000)
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:5000/\"".format(public_url))

app.run(port=5000)
